In [497]:
import pandas as pd
import numpy as np
from sklearn import preprocessing

In [498]:
train_df_raw = pd.read_csv("./data/training.csv")

In [499]:
routes_df = pd.read_csv("./data/routes.csv")
route_list = routes_df['BUSROUTE_ID'].unique()

In [500]:
stop_df = pd.read_csv("./data/stops.csv")
stop_list = stop_df['BUSSTOP_ID'].unique()

In [501]:
bus_list = train_df_raw['BUS_ID'].unique()

In [502]:
train_df = train_df_raw

In [503]:
train_df.head()

,index,RECORD_DATE,BUS_ID,BUSROUTE_ID,BUSSTOP_ID,BUSSTOP_SEQ,TIMESTAMP
0,0,2020-01-06 07:00:00,325819008,11100360,388,21,1578294000
1,1,2020-01-06 07:00:00,451509115,11100480,112,9,1578294000
2,2,2020-01-06 07:00:00,235501055,11100500,374,9,1578294000
3,3,2020-01-06 07:00:00,145010001,11100330,331,30,1578294000
4,4,2020-01-06 07:00:01,421733004,11100620,51,29,1578294001


In [504]:
train_df = train_df.sort_values(by=['BUS_ID', 'TIMESTAMP'])

In [505]:
train_df['TIMESTAMP_DIFF'] = train_df['TIMESTAMP'].shift(-1) - train_df['TIMESTAMP']

In [506]:
train_df['SEQ_DIFF'] = train_df['BUSSTOP_SEQ'].shift(-1) - train_df['BUSSTOP_SEQ']

In [507]:
train_df['ROUTE_DIFF'] = train_df['BUSROUTE_ID'].shift(-1) - train_df['BUSROUTE_ID']

In [508]:
train_df['BUS_ID_DIFF'] = train_df['BUS_ID'].shift(-1) - train_df['BUS_ID']

In [509]:
# train_df = train_df[(train_df['SEQ_DIFF']==1.0) & (train_df['ROUTE_DIFF']==0.0) & (train_df['BUS_ID_DIFF']==0.0)]

In [510]:
train_df = train_df[train_df['SEQ_DIFF']==1]
train_df = train_df[train_df['ROUTE_DIFF']==0]
train_df = train_df[train_df['BUS_ID_DIFF']==0]

In [511]:
train_df[train_df['TIMESTAMP_DIFF']<1]

,index,RECORD_DATE,BUS_ID,BUSROUTE_ID,BUSSTOP_ID,BUSSTOP_SEQ,TIMESTAMP,TIMESTAMP_DIFF,SEQ_DIFF,ROUTE_DIFF,BUS_ID_DIFF


In [512]:
# bus id encoding
bus_id_encoder = preprocessing.LabelEncoder()
bus_id_encoder.fit(bus_list)
train_df.BUS_ID = bus_id_encoder.transform(train_df.BUS_ID)

# bus_id_encoder.transform([145010001, 235501055])
# bus_id_encoder.inverse_transform([280, 466])

In [513]:
# route id encoding
route_id_encoder = preprocessing.LabelEncoder()
route_id_encoder.fit(route_list)
train_df.BUSROUTE_ID = route_id_encoder.transform(train_df.BUSROUTE_ID)

In [514]:
# stop id encoding
stop_id_encoder = preprocessing.LabelEncoder()
stop_id_encoder.fit(stop_list)
train_df.BUSSTOP_ID = stop_id_encoder.transform(train_df.BUSSTOP_ID)

In [515]:
train_df

,index,RECORD_DATE,BUS_ID,BUSROUTE_ID,BUSSTOP_ID,BUSSTOP_SEQ,TIMESTAMP,TIMESTAMP_DIFF,SEQ_DIFF,ROUTE_DIFF,BUS_ID_DIFF
1624542,1624542,2020-01-13 07:28:25,0,3,17,2,1578900505,117.0,1.0,0.0,0.0
1625151,1625151,2020-01-13 07:30:22,0,3,10,3,1578900622,123.0,1.0,0.0,0.0
1625791,1625791,2020-01-13 07:32:25,0,3,8,4,1578900745,121.0,1.0,0.0,0.0
1626425,1626425,2020-01-13 07:34:26,0,3,5,5,1578900866,136.0,1.0,0.0,0.0
1627161,1627161,2020-01-13 07:36:42,0,3,3,6,1578901002,174.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
2893774,2893774,2020-01-17 21:47:30,1076,8,109,31,1579297650,74.0,1.0,0.0,0.0
2894017,2894017,2020-01-17 21:48:44,1076,8,117,32,1579297724,123.0,1.0,0.0,0.0
2894432,2894432,2020-01-17 21:50:47,1076,8,119,33,1579297847,91.0,1.0,0.0,0.0
2894723,2894723,2020-01-17 21:52:18,1076,8,121,34,1579297938,82.0,1.0,0.0,0.0


In [516]:
train_df['DAY_OF_WEEK'] = pd.to_datetime(train_df['RECORD_DATE']).dt.dayofweek

In [517]:
# time validation (valid time is 07:00:00 ~ 23:59:59)
train_df['TIME_validation'] = train_df['TIMESTAMP'] % 86400
train_df = train_df[train_df['TIME_validation']>=25200]

In [518]:
# encodeing time

# (pd.to_datetime(train_df['RECORD_DATE']) - pd.Timestamp("1970-01-01")) // pd.Timedelta('1s')
# (pd.Timestamp('2020-01-14 07:00:00')  - pd.Timestamp("1970-01-01")) // pd.Timedelta('1s')
train_df['TIME'] = (train_df['TIMESTAMP'] - 25200) % 86400


/Users/zolbayar/code/kaggle-datanomads-bus-arrival/env/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [519]:
train_df.sort_values(by=['TIMESTAMP_DIFF'])

,index,RECORD_DATE,BUS_ID,BUSROUTE_ID,BUSSTOP_ID,BUSSTOP_SEQ,TIMESTAMP,TIMESTAMP_DIFF,SEQ_DIFF,ROUTE_DIFF,BUS_ID_DIFF,DAY_OF_WEEK,TIME_validation,TIME
2495182,2495182,2020-01-16 13:20:24,610,206,378,16,1579180824,3.0,1.0,0.0,0.0,3,48024,22824
257759,257759,2020-01-07 07:23:00,296,260,1170,47,1578381780,4.0,1.0,0.0,0.0,1,26580,1380
80314,80314,2020-01-06 11:18:34,725,251,379,27,1578309514,4.0,1.0,0.0,0.0,0,40714,15514
2543779,2543779,2020-01-16 16:05:54,200,251,379,27,1579190754,5.0,1.0,0.0,0.0,3,57954,32754
1170937,1170937,2020-01-10 14:41:21,427,41,452,40,1578667281,5.0,1.0,0.0,0.0,4,52881,27681
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1274219,1274219,2020-01-10 21:28:38,710,135,163,8,1578691718,207631.0,1.0,0.0,0.0,4,77318,52118
1031576,1031576,2020-01-09 22:09:17,468,200,1158,21,1578607757,208388.0,1.0,0.0,0.0,3,79757,54557
1238330,1238330,2020-01-10 19:06:14,219,265,148,13,1578683174,214548.0,1.0,0.0,0.0,4,68774,43574
1162737,1162737,2020-01-10 14:12:16,800,223,1194,25,1578665536,232925.0,1.0,0.0,0.0,4,51136,25936


In [520]:
# 2000 sec-ees ihiig evdersen esvel tur zogsolt hiisen gej uzeed haslaa

# train_df[(train_df['TIMESTAMP_DIFF']>2000)].sort_values(by=['TIMESTAMP_DIFF'])
train_df = train_df[(train_df['TIMESTAMP_DIFF']<2000)]

In [521]:
train_df

,index,RECORD_DATE,BUS_ID,BUSROUTE_ID,BUSSTOP_ID,BUSSTOP_SEQ,TIMESTAMP,TIMESTAMP_DIFF,SEQ_DIFF,ROUTE_DIFF,BUS_ID_DIFF,DAY_OF_WEEK,TIME_validation,TIME
1624542,1624542,2020-01-13 07:28:25,0,3,17,2,1578900505,117.0,1.0,0.0,0.0,0,26905,1705
1625151,1625151,2020-01-13 07:30:22,0,3,10,3,1578900622,123.0,1.0,0.0,0.0,0,27022,1822
1625791,1625791,2020-01-13 07:32:25,0,3,8,4,1578900745,121.0,1.0,0.0,0.0,0,27145,1945
1626425,1626425,2020-01-13 07:34:26,0,3,5,5,1578900866,136.0,1.0,0.0,0.0,0,27266,2066
1627161,1627161,2020-01-13 07:36:42,0,3,3,6,1578901002,174.0,1.0,0.0,0.0,0,27402,2202
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2893774,2893774,2020-01-17 21:47:30,1076,8,109,31,1579297650,74.0,1.0,0.0,0.0,4,78450,53250
2894017,2894017,2020-01-17 21:48:44,1076,8,117,32,1579297724,123.0,1.0,0.0,0.0,4,78524,53324
2894432,2894432,2020-01-17 21:50:47,1076,8,119,33,1579297847,91.0,1.0,0.0,0.0,4,78647,53447
2894723,2894723,2020-01-17 21:52:18,1076,8,121,34,1579297938,82.0,1.0,0.0,0.0,4,78738,53538


In [526]:
output_df = train_df[["DAY_OF_WEEK", "TIME", "BUS_ID", "BUSROUTE_ID", "BUSSTOP_ID"]]

In [527]:
output_df.to_csv('cleaned_train.csv', index = False)


In [65]:
train_df.groupby('BUSROUTE_ID').agg({'BUSROUTE_ID':'size', 'TIMESTAMP_DIFF':'mean'}) \
.rename(columns={'BUSROUTE_ID':'count','TIMESTAMP_DIFF':'mean_sent'}).sort_values(by=['mean_sent'])

,count,mean_sent
BUSROUTE_ID,,
11100680,5781,-88.386784
11400050,1,50.000000
11400070,24360,77.556363
11100631,2,90.500000
11400110,6,92.166667
...,...,...
11100600,1250,291.690400
11400120,4238,300.787636
11100040,21092,391.180542


In [41]:
print(len(train_df_raw))
print(len(train_df))

3220254
3116418


In [483]:
one_bus  = train_df[train_df['BUS_ID']==800]

In [484]:
one_bus

,index,RECORD_DATE,BUS_ID,BUSROUTE_ID,BUSSTOP_ID,BUSSTOP_SEQ,TIMESTAMP,TIMESTAMP_DIFF,SEQ_DIFF,ROUTE_DIFF,BUS_ID_DIFF,DAY_OF_WEEK,TIME_validation,TIME
2170,2170,2020-01-06 07:10:50,800,223,18,27,1578294650,136.0,1.0,0.0,0.0,0,25850,650
2696,2696,2020-01-06 07:13:06,800,223,11,28,1578294786,115.0,1.0,0.0,0.0,0,25986,786
3153,3153,2020-01-06 07:15:01,800,223,9,29,1578294901,112.0,1.0,0.0,0.0,0,26101,901
3670,3670,2020-01-06 07:16:53,800,223,6,30,1578295013,107.0,1.0,0.0,0.0,0,26213,1013
4142,4142,2020-01-06 07:18:40,800,223,4,31,1578295120,165.0,1.0,0.0,0.0,0,26320,1120
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2604155,2604155,2020-01-16 19:55:23,800,223,5,20,1579204523,199.0,1.0,0.0,0.0,3,71723,46523
2605118,2605118,2020-01-16 19:58:42,800,223,7,21,1579204722,131.0,1.0,0.0,0.0,3,71922,46722
2605729,2605729,2020-01-16 20:00:53,800,223,8,22,1579204853,170.0,1.0,0.0,0.0,3,72053,46853
2606526,2606526,2020-01-16 20:03:43,800,223,10,23,1579205023,173.0,1.0,0.0,0.0,3,72223,47023


In [485]:
one_bus.to_excel("output.xlsx")

In [30]:
one_bus = one_bus.dropna()

In [36]:
one_bus

,index,RECORD_DATE,BUS_ID,BUSROUTE_ID,BUSSTOP_ID,BUSSTOP_SEQ,TIMESTAMP,TIMESTAMP_DIFF
0,0,2020-01-06 07:00:00,325819008,11100360,388,21,1578294000,66.0
191,191,2020-01-06 07:01:06,325819008,11100360,386,22,1578294066,101.0
492,492,2020-01-06 07:02:47,325819008,11100360,384,23,1578294167,116.0
878,878,2020-01-06 07:04:43,325819008,11100360,382,24,1578294283,94.0
1170,1170,2020-01-06 07:06:17,325819008,11100360,1402,25,1578294377,87.0
...,...,...,...,...,...,...,...,...
3218517,3218517,2020-01-19 22:09:41,325819008,11100360,1402,25,1579471781,66.0
3218597,3218597,2020-01-19 22:10:47,325819008,11100360,380,26,1579471847,79.0
3218695,3218695,2020-01-19 22:12:06,325819008,11100360,378,27,1579471926,62.0
3218779,3218779,2020-01-19 22:13:08,325819008,11100360,376,28,1579471988,300.0
